### PaliGemma Pipeline1

In [ ]:
!nvidia-smi

Sun Dec  1 21:28:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 32.6/235.7 GB disk)


In [ ]:
!pip install -U bitsandbytes
!pip install accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
import os

# Replace "your_token_here" with your actual Hugging Face token
hf_token = os.environ.get("HF_TOKEN")
login(token=hf_token)


In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
import requests
from PIL import Image
import copy
import torch
import shutil
%matplotlib inline

In [ ]:
# model_id = "google/paligemma-3b-mix-224"

# pali_model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).eval()
# processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
model_id = "google/paligemma-3b-mix-224"
device = "cuda:0"
dtype = torch.bfloat16

pali_model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map=device,
    revision="bfloat16",
).eval()
pali_processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

#### PaliGemma Inference Function

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
def run_pali_example(image):
    # Define a prompt with image tokens and <bos> token
    prompt = "<image><bos> very short caption"

    # Process the input for the model
    model_inputs = pali_processor(text=prompt, images=image, return_tensors="pt")

    # Move tensors to model's device
    model_inputs = {
        k: v.to(pali_model.device, dtype=dtype if v.dtype == torch.float else torch.long)
        for k, v in model_inputs.items()
    }

    # Get input length for decoding
    input_len = model_inputs["input_ids"].shape[-1]

    # Generate output
    with torch.inference_mode():
        generation = pali_model.generate(**model_inputs, max_new_tokens=70, do_sample=False)
        generation = generation[0][input_len:]
        decoded = pali_processor.decode(generation, skip_special_tokens=True)

    return decoded

In [ ]:
# image = Image.open('/content/video_frames/1. a black car is moving on empty road/00000.png')
run_pali_example(image)

'In this image we can see a car on the road. In the background there is a wall, door and trees.'

In [ ]:
# def run_pali_example(image):
#   # Instruct the model to create a caption; Ex: task_prompt = 'short caption'
#   model_inputs = processor(text='very short caption', images=image, return_tensors="pt")
#   input_len = model_inputs["input_ids"].shape[-1]

#   with torch.inference_mode():
#       generation = pali_model.generate(**model_inputs, max_new_tokens=70, do_sample=False)
#       generation = generation[0][input_len:]
#       decoded = processor.decode(generation, skip_special_tokens=True)
#   return decoded

In [ ]:
# Clear CUDA cache (if using PyTorch)
torch.cuda.empty_cache()

In [ ]:
# Florence run_example() function

# def run_example(task_prompt, image, text_input=None):
#     if text_input is None:
#         prompt = task_prompt
#     else:
#         prompt = task_prompt + text_input
#     inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
#     generated_ids = flor_model.generate(
#       input_ids=inputs["input_ids"].cuda(),
#       pixel_values=inputs["pixel_values"].cuda(),
#       max_new_tokens=1024,
#       early_stopping=False,
#       do_sample=False,
#       num_beams=3,
#     )
#     generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
#     parsed_answer = processor.post_process_generation(
#         generated_text,
#         task=task_prompt,
#         image_size=(image.width, image.height)
#     )

#     return parsed_answer

#### Generate Frames from Video

In [ ]:
import os
import cv2

# Specify the directory path
directory_path = '/content/FateZero'

# Get a list of all file names with their full paths
file_paths = [os.path.join(directory_path, file_name) for file_name in os.listdir(directory_path)]

# # Print the paths of each fle
# for file_name in file_paths:
#     print(file_name)

In [ ]:
def create_frames(video_paths, output_dir):
  # Clear the directory by deleting all its contents
  if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
  for path in video_paths:
    # Path to the video file
    editing_prompt = path.split('/')[-1][:-4]

    # Directory where frames will be saved
    output_dir = '/content/video_frames/{editing_prompt}'.format(editing_prompt=editing_prompt)

    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(path)

    frame_count = 0
    while True:
        ret, frame = cap.read()
        # If there are no more frames, break the loop
        if not ret:
            break

        # Resize each frame to 512x512
        frame_resized = cv2.resize(frame, (512, 512))
        # Save each frame as a PNG file
        frame_name = os.path.join(output_dir, f'{frame_count:05d}.png')
        cv2.imwrite(frame_name, frame_resized)
        frame_count += 1

    # Release the video capture object
    cap.release()
    print(f"Extracted {frame_count} frames and saved them to {output_dir}")

In [ ]:
output_dir = '/content/video_frames'
create_frames(file_paths, output_dir)

Extracted 16 frames and saved them to /content/video_frames/8. a fish is swimming in empty ocean
Extracted 16 frames and saved them to /content/video_frames/26. an octopus is diving under water in search of food
Extracted 16 frames and saved them to /content/video_frames/13. a porcupine is walking on the ground
Extracted 16 frames and saved them to /content/video_frames/19. a tiger is walking on the ground
Extracted 16 frames and saved them to /content/video_frames/16. a seagull is diving under water insearch of food
Extracted 16 frames and saved them to /content/video_frames/11. a mouse is walking on the ground
Extracted 16 frames and saved them to /content/video_frames/1. a black car is moving on empty road
Extracted 16 frames and saved them to /content/video_frames/9. a lion is running on the ground
Extracted 16 frames and saved them to /content/video_frames/7. a dolphin is swimming under water in search of food
Extracted 16 frames and saved them to /content/video_frames/5. a dog is

#### Text Similarity Function

In [ ]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tn55yx9t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tn55yx9t
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=1ff88dec2e738902ca63557310f1287706a4218e91759b9f591e918692e06cf9
  Stored in directory: /tmp/pip-ephem-wheel-cache-pzur2phl/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
import torch
import clip

# Load the CLIP model once outside the function
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_flor_model, _ = clip.load("ViT-B/32", device=device)

def clip_text_similarity(prompt1, prompt2, model):
    # Tokenize the prompts and encode them into embeddings
    text_inputs = clip.tokenize([prompt1, prompt2]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)

    # Normalize the embeddings to compute cosine similarity
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity between the two prompts
    similarity_score = (text_features @ text_features.T).cpu().numpy()

    return similarity_score[0][1]

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 124MiB/s]


### Grounding-DINO Pipeline

In [ ]:
import os
HOME = os.getcwd()
print(HOME)
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content
/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 463 (delta 175), reused 135 (delta 135), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 34.42 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/content
/content/weights
/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

dino_model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### LLM Classifier with Few-Shot Prompting

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

llm_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
test_prompt = 'a dolphin is swimming in the coral reef'

def extract_primary_object(prompt, llm_model):

  user_prompt = f"""
  Extract the primary object mentioned in the sentence.
  The primary object is the main noun that represents the entity performing or being described.
  Return only that object and nothing else.

  Example:
  Input: "A cat is sitting on the window."
  Output: "cat"
  Input: "A white car is moving on the road."
  Output: "white car"
  Input: "chnage the cat into lion"
  Output: "lion"
  Now, detect the primary object in the following sentence:

  Input: {test_prompt}
  Final Output:
  """
  messages = [
      {"role": "user", "content": user_prompt}
  ]

  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

  outputs = llm_model.generate(inputs, max_new_tokens=20)
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # Clean up the output
  response = response.split("Final Output:")[1].strip().replace('"', '')
  return response

object_detected = extract_primary_object(test_prompt, llm_model)
print('========================')
print('Primary Object Detected:', object_detected)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Primary Object Detected: dolphin


In [ ]:
import os
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor for cosine similarity
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
# Function to compute cosine similarity using CLIP embeddings
def compute_clip_similarity_od(phrase1, phrase2):
    inputs = clip_processor(text=[phrase1, phrase2], return_tensors="pt", padding=True).to("cuda")
    with torch.no_grad():
        embeddings = clip_model.get_text_features(**inputs)
    similarity = cosine_similarity(embeddings.cpu().numpy())
    return similarity[0][1]

### Loop to calculate all below scores:
1. PaliGemma (Semantic Text Scores)
2. Grounding DINO (Object Detection Scores)
3. Temporal Consistency Score

In [ ]:
import warnings
import torch
import numpy as np

# Ignore all warnings
warnings.filterwarnings("ignore")
torch.cuda.empty_cache()


In [ ]:
# Function to compute CLIP similarity between two images (for temporal consistency)
def compute_clip_similarity(image1, image2):
    inputs = clip_processor(images=[image1, image2], return_tensors="pt", padding=True).to('cuda')
    with torch.no_grad():
        image_embeddings = clip_model.get_image_features(**inputs)

    cosine_similarity = torch.nn.functional.cosine_similarity(image_embeddings[0], image_embeddings[1], dim=0)
    return cosine_similarity.item()

# Function to calculate temporal consistency score for a video
def calculate_temporal_consistency(video_frames_path):
    frames = []
    for filename in sorted(os.listdir(video_frames_path)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(video_frames_path, filename)).convert("RGB")
            frames.append(img)

    # Calculate similarity between consecutive frames
    similarities = []
    for i in range(len(frames) - 1):
        sim = compute_clip_similarity(frames[i], frames[i + 1])
        similarities.append(sim)

    # Calculate temporal consistency as the average similarity
    if similarities:
        temporal_consistency = np.mean(similarities)
    else:
        temporal_consistency = 0.0

    return temporal_consistency

In [ ]:
import os
import pandas as pd
from PIL import Image

# List to store the results for the dataframe
data = []

# Specify the root directory containing the folders with frames
root_dir = '/content/video_frames'

# Define parameters for object detection
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.25
MIN_CONFIDENCE_SCORE = 0.1  # Minimum confidence score to be added when no detections

# Iterate over all folders in the root directory
for folder_name in sorted(os.listdir(root_dir)):
    folder_path = os.path.join(root_dir, folder_name)
    editing_prompt = folder_name
    # Check if it is a directory
    if os.path.isdir(folder_path):
        # List to store similarity scores for each frame in the folder
        similarity_scores = []
         # List to store object detection results for each frame
        results = []
        # Iterate over all frames in each folder
        for frame_file in sorted(os.listdir(folder_path)):
            # Ensure we are working with image files only (skip other files)
            if frame_file.endswith(('.png', '.jpg', '.jpeg')):
                # Load the image
                image_path = os.path.join(folder_path, frame_file)
                image = Image.open(image_path)

                # Generate Florence caption for the image
                flor_caption = run_pali_example( image)

                # Calculate the similarity score between the original prompt and Florence caption
                similarity = clip_text_similarity(editing_prompt, flor_caption, clip_flor_model)
                # Append the similarity score to the list
                similarity_scores.append(similarity)

                # --- Object detection logic starts here ---

                # Load the image and predict the boxes, logits, and phrases
                image_source, image = load_image(image_path)
                boxes, logits, phrases = predict(
                    model=dino_model,
                    image=image,
                    caption=editing_prompt,
                    box_threshold=BOX_THRESHOLD,
                    text_threshold=TEXT_THRESHOLD
                )

                # Extract confidence scores
                if logits.ndim == 1:  # If logits is 1D, it means there is only one class
                    confidence_scores = logits  # Confidence scores are the logits themselves
                elif logits.ndim == 2:  # If logits is 2D, we can get the max across classes
                    confidence_scores = logits.max(axis=1)  # Get max score for each box
                else:
                    print(f"Unexpected logits shape for image {frame_file}: {logits.shape}")
                    continue  # Skip to the next image if the shape is unexpected
                if len(boxes) == 0:
                    # If no detections, set minimum confidence score
                    confidence = MIN_CONFIDENCE_SCORE
                    phrase = "No detections"
                else:
                  # Check if there are multiple detections, if so, filter by cosine similarity
                  if len(phrases) > 1:
                      # Extract the primary object from the prompt using LLM
                      primary_object = extract_primary_object(editing_prompt, llm_model)
                      # Compute cosine similarity between each detected object and the primary object
                      similarity_scores_phrases = [
                          compute_clip_similarity_od(phrase, primary_object) for phrase in phrases
                      ]
                      # Find the detection with the highest similarity score
                      max_similarity_idx = similarity_scores_phrases.index(max(similarity_scores_phrases))
                      # Select the corresponding box, confidence, and phrase
                      box = boxes[max_similarity_idx]
                      confidence = confidence_scores[max_similarity_idx].item()
                      phrase = phrases[max_similarity_idx]
                  else:
                      # If only one detection, take it directly
                      box = boxes[0]
                      confidence = confidence_scores[0].item()
                      phrase = phrases[0]

                # Store the results of object detection
                results.append({
                    'folder': folder_name,
                    'image_name': frame_file,
                    'detected_object': phrase,
                    'confidence_score': confidence  # Confidence score of the object detection
                })

        # --- Object detection logic ends here ---

        # Calculate the mean similarity score for the current folder (video)
        if similarity_scores:
            mean_similarity = sum(similarity_scores) / len(similarity_scores)

            # Calculate mean object detection confidence score for the current folder
            if results:
                object_detection_score = sum([r['confidence_score'] for r in results]) / len(results)
            else:
                object_detection_score = 0.0
            # --- Calculate Temporal Consistency ---
            temporal_consistency_score = calculate_temporal_consistency(folder_path)

            # Store the mean similarity score and object detection score with the video (folder) name
            data.append({
                'video': folder_name,   # Store the video name (folder name)
                'Sematic_Similarity_Score': mean_similarity,
                'Object_Detection_score': object_detection_score,
                'temporal_consistency_score': temporal_consistency_score

            })

# Create a dataframe with the collected data
df = pd.DataFrame(data)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
import pandas as pd
from google.colab import files

# Assuming 'df' is your DataFrame
df.to_csv('fatezero_custom_metric_results.csv', index=False)  # Save DataFrame as CSV

# Download the file
files.download('fatezero_custom_metric_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.cuda.empty_cache()

## **Custom Metric Formulas**
### Formula 1:
$$
S_{\text{final}} = w_1 \times \frac{1}{n} \sum_{i=1}^{n} S_{\text{context_similarity}}^{(i)} + w_2 \times \frac{1}{n} \sum_{i=1}^{n} S_{\text{object_detection}}^{(i)} + w_3 \times \left( 1 - \frac{1}{n} \sum_{i=1}^{n} S_{\text{temporal_consistency}}^{(i)} \right)
$$
### Formula 2:
$$
S_{\text{final}} = w \times \left( \frac{1}{n} \sum_{i=1}^{n} S_{\text{context_similarity}}^{(i)} \right) \times \left( \frac{1}{n} \sum_{i=1}^{n} S_{\text{object_detection}}^{(i)} \right) \times \left( \frac{1}{n} \sum_{i=1}^{n} S_{\text{temporal_consistency}}^{(i)} \right)
$$



In [ ]:
import pandas as pd
# define final metrics using the calculated metrics so far
def final_score(context_similarity_score, object_detection_score, temporal_consistency_score):
  # weights can be adjusted according to the imprtance of each
  w1 = 0.361 # Weights finalised from regression using Human Eval
  w2 = 0.136
  w3 = 0.502
  # make sure sum of weights is eual to one
  # consider normalising the score values beofre using into the formula
  S_similarity = context_similarity_score
  S_object_detection = object_detection_score
  S_temporal = temporal_consistency_score
  S_final1 = w1 * S_similarity + w2 * S_object_detection + w3 * S_temporal
  S_final2 = S_similarity*S_object_detection*S_temporal
  return S_final1

In [ ]:
df = pd.read_csv('/content/controlavideo_custom_metric_results.csv')
df.head()

,video,Sematic_Similarity_Score,Object_Detection_score,temporal_consistency_score
0,1. a black car is moving on empty road,0.741943,0.933479,0.954393
1,10. a lion is walking on the ground,0.749442,0.928618,0.954056
2,11. a mouse is walking on the ground,0.570714,0.755483,0.914044
3,12. a pigeon is swimming in the water,0.808454,0.466809,0.951793
4,13. a porcupine is walking on the ground,0.591221,0.737835,0.919784


In [ ]:
# Function to apply the final_score function to each row
def calculate_final_scores(row):
    return final_score(row['Sematic_Similarity_Score'], row['Object_Detection_score'], row['temporal_consistency_score'])

# Apply the function and create new columns for final scores
df['Final_score'] = df.apply(calculate_final_scores, axis=1, result_type='expand')

# Display the updated DataFrame with final scores
df.head()

,video,Sematic_Similarity_Score,Object_Detection_score,temporal_consistency_score,Final_score
0,1. a black car is moving on empty road,0.741943,0.933479,0.954393,0.873900
1,10. a lion is walking on the ground,0.749442,0.928618,0.954056,0.875777
2,11. a mouse is walking on the ground,0.570714,0.755483,0.914044,0.767624
3,12. a pigeon is swimming in the water,0.808454,0.466809,0.951793,0.833138
4,13. a porcupine is walking on the ground,0.591221,0.737835,0.919784,0.775508


In [ ]:
df.to_csv('controlavideo_custom_metric_results_final.csv', index=False)
# Download the file
files.download('controlavideo_custom_metric_results_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Fitting Regression Model to estimate w1, w2 and w3 values.

In [ ]:
# import pandas as pd
# import numpy as np

# # Set the random seed for reproducibility
# np.random.seed(42)
# # Length of the DataFrame
# length = len(df)
# # Generate random values for final_score
# final_score = np.random.uniform(0.5, 1, length)
# # Create the DataFrame
# df["human_final_score"] = final_score
# # # Display the DataFrame
# # df.head()

In [ ]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# import numpy as np

# # Prepare features and target
# X = df[['Sematic_Similarity_Score', 'Object_Detection_score', 'temporal_consistency_score']]
# y = df['human_final_score']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create and fit the regression model
# LR_model = LinearRegression()
# LR_model.fit(X_train, y_train)

# # Get the weights (coefficients) for w1, w2, w3
# weights = LR_model.coef_
# intercept = LR_model.intercept_

# w1, w2, w3 = weights
# print("Weights:", w1, w2, w3)

In [ ]:
# # Apply softmax normalization
# def softmax(x):
#     e_x = np.exp(x - np.max(x))  # Subtract max to avoid overflow
#     return e_x / e_x.sum()

# # Normalize raw weights using softmax
# normalized_weights = softmax(weights)
# w1, w2, w3 = normalized_weights
# print("Normalized Weights:")
# print(f"w1: {w1}")
# print(f"w2: {w2}")
# print(f"w3: {w3}")

In [ ]:
# # define final metrics using the calculated metrics so far
# def final_score(context_similarity_score, object_detection_score, temporal_consistency_score, weights):
#   # weights can be adjusted according to the imprtance of each
#   w = 1
#   w1, w2, w3 = weights
#   # make sure sum of weights is eual to one
#   # consider normalising the score values beofre using into the formula
#   S_similarity = context_similarity_score
#   S_object_detection = object_detection_score
#   S_temporal = temporal_consistency_score
#   S_final1 = w1 * S_similarity + w2 * S_object_detection + w3 * (1 - S_temporal)
#   S_final2 = w*S_similarity*S_object_detection*S_temporal
#   return S_final1, S_final2

In [ ]:
# # Function to apply the final_score function to each row
# def calculate_final_scores(row):
#     return final_score(row['Sematic_Similarity_Score'], row['Object_Detection_score'], row['temporal_consistency_score'], normalized_weights)

# # Apply the function and create new columns for final scores
# df[['final_score_1', 'final_score_2']] = df.apply(calculate_final_scores, axis=1, result_type='expand')

# # Display the updated DataFrame with final scores
# df.head()